Задание 9. Решение задачи теплопроводности сеточными методами (явная и неявная схемы).

In [1]:
from dataclasses import dataclass
from enum import Enum
from math import cos
from typing import Callable

import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from scipy.sparse import lil_matrix

In [2]:
Function = Callable[[float], float]
BinaryFunction = Callable[[float, float], float]
HeatEquationSolution = tuple[np.ndarray, np.ndarray, np.ndarray]


@dataclass
class HeatEquation:
    '''
    Form of the equation:

    ``uₜ(x, t) = kuₓₓ(x, t) + f(x, t)``,

    Initial condition:

    ``u(x, 0) = μ(x)``,

    Boundary conditions:

    ``u(0, t) = μ₁(t), u(a, t) = μ₂(t)``
    '''
    k: float
    f: BinaryFunction
    right_x: float
    right_t: float
    initial_condition: Function
    l_boundary_condition: Function
    r_boundary_condition: Function


class Scheme(Enum):
    EXPLICIT = 0
    IMPLICIT = 1

In [3]:
def solve_tridiagonal_system(matrix: lil_matrix, b: np.ndarray) -> np.ndarray:
    s = np.zeros(b.size)
    t = np.zeros(b.size)
    A, B, C, G = np.insert(matrix.diagonal(-1), 0, 0), -matrix.diagonal(), np.append(matrix.diagonal(1), 0), b

    for i in range(b.size):
        s[i] = C[i] / (B[i] - A[i] * s[i - 1])
        t[i] = (A[i] * t[i - 1] - G[i]) / (B[i] - A[i] * s[i - 1])
    
    y = np.zeros(b.size)
    y[-1] = t[-1]

    for i in reversed(range(b.size - 1)):
        y[i] = s[i] * y[i + 1] + t[i]

    return y


def solve_heat_equation(equation: HeatEquation, num_x: int, num_t: int, scheme: Scheme) -> HeatEquationSolution:
    x, step_x = np.linspace(0, equation.right_x, num_x, retstep=True)
    t, step_t = np.linspace(0, equation.right_t, num_t, retstep=True)
    
    u = np.zeros((t.size, x.size))
    u[0] = np.vectorize(equation.initial_condition)(x)

    coeff = step_t * equation.k / (step_x ** 2)
    for i in range(1, t.size):
        match scheme:
            case Scheme.IMPLICIT:
                matrix = lil_matrix((x.size, x.size))
                b = np.zeros(x.size)

                for j in range(1, x.size - 1):
                    matrix[j, j - 1] = -coeff
                    matrix[j, j] = 2 * coeff + 1
                    matrix[j, j + 1] = -coeff
                    b[j] = step_t * equation.f(x[j], t[i]) + u[i - 1, j]

                matrix[0, 0], matrix[-1, -1] = 1, 1
                b[0], b[-1] = equation.l_boundary_condition(t[i]), equation.r_boundary_condition(t[i])

                u[i] = solve_tridiagonal_system(matrix, b)

            case Scheme.EXPLICIT:
                u[i, 0] = equation.l_boundary_condition(t[i])
                u[i, -1] = equation.r_boundary_condition(t[i])
                for j in range(1, x.size - 1):
                    prev_u = u[i - 1]
                    u[i, j] = (prev_u[j] + 
                               coeff * (prev_u[j - 1] - 2 * prev_u[j] + prev_u[j + 1]) + 
                               step_t * equation.f(x[j], t[i - 1]))
    return x, t, u

In [4]:
u = lambda x, t: t ** 2 * cos(x) + 0.5 * x ** 2
u_t = lambda x, t: 2 * t * cos(x)
u_xx = lambda x, t: t ** 2 * -cos(x) + 1
k = 2
f = lambda x, t: u_t(x, t) - k * u_xx(x, t)

right_x = 1
right_t = 1

initial_condition = lambda x: u(x, 0)
l_condition = lambda t: u(0, t)
r_condition = lambda t: u(right_x, t)

equation = HeatEquation(k, f, right_x, right_t, initial_condition, l_condition, r_condition)

fig = make_subplots(rows=2, cols=2,
                    specs=[[{'is_3d': True}, {'is_3d': True}],
                           [{'is_3d': True}, {'is_3d': True}]], 
                    vertical_spacing=0.15,
                    subplot_titles=['Точное решение', 'Неявная схема',
                                    'Явная схема (условие не выполнено)', 'Явная схема (условие выполнено)'])

num_x, num_t = 11, 11
step_x = (equation.right_x - 0) / (num_x - 1)

x, t, sol_u = solve_heat_equation(equation, num_x, num_t, Scheme.IMPLICIT)
fig.append_trace(go.Surface(x=x, y=t, z=sol_u, showscale=False), row=1, col=2)

grid = np.meshgrid(x, t)
exact_u = np.vectorize(u)(*grid)
fig.append_trace(go.Surface(x=x, y=t, z=exact_u, colorscale='tropic', showscale=False), row=1, col=1)

step_t = step_x ** 2 / equation.k / 4
num_t = round((equation.right_t - 0) / step_t) + 1
x, t, sol_u = solve_heat_equation(equation, num_x, num_t, Scheme.EXPLICIT)
fig.append_trace(go.Surface(x=x, y=t, z=sol_u, showscale=False), row=2, col=2)

step_t = step_x ** 2 / equation.k
num_t = round((equation.right_t - 0) / step_t) + 1
x, t, sol_u = solve_heat_equation(equation, num_x, num_t, Scheme.EXPLICIT)
fig.append_trace(go.Surface(x=x, y=t, z=sol_u, showscale=False), row=2, col=1)

fig.update_scenes(yaxis_title='t', zaxis_title='u(x, t)')
fig.update_scenes(zaxis_tickformat='.0e', row=2, col=1)
fig.update_traces(hovertemplate='x: %{x}<br>t: %{y}<br>u(x, t): %{z}<extra></extra>')
fig.update_layout(height=600, margin=dict(r=10, l=10, b=30, t=30))

fig.show()